# Introduction

In [ ]:
# preliminaries - import libraries and classes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.options.display.max_columns = 999

In [ ]:
# read-in data
df = pd.read_csv('AmesHousing.tsv', delimiter='\t')

In [ ]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

# Feature Engineering

In general, the goal of this function is to:
- remove features that we don't want to use in the model, just based on the number of missing values or data leakage
- transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc.)
- create new features by combining other features

#### Missing Values
- For all columns: drop any with 5% or more missing values
- For text columns: drop any with 1 or more missing values
- For numerical columns: fill in missing values with mode

1: All values: drop any with %5 or more missing values

In [ ]:
# identify features with missing values
#num_missing = df.isnull().sum()
#num_missing

In [ ]:
# filter out those with more than 5% missing
#drop_missing_cols = num_missing[(num_missing > len(df)*.05)].sort_values()

# drop those filters from the dataframe
#df = df.drop(drop_missing_cols.index, axis=1)

2: Text Values: drop any missing 1 or more values

In [ ]:
# identify text features with missing values
#text_mv_count = df.select_dtypes(include='object').isnull().sum().sort_values(ascending=False)
#text_mv_count

In [ ]:
# filter out those with one or more missing
#drop_missing_cols_2 = text_mv_count[(text_mv_count > 0)]

# drop these from the data frame
#df = df.drop(drop_missing_cols_2.index, axis=1)

3: Numerical Values: where less than 5% are missing, fill in missing values with mode

In [ ]:
# identify numerical columns with missing values
#num_missing = df.select_dtypes(include=['integer','float']).isnull().sum()
#num_missing

In [ ]:
# filter those with less than 5% missing and impute with mode
#fixable_num_cols = num_missing[(num_missing < len(df)/20) & (num_missing >0)].sort_values()
#fixable_num_cols

In [ ]:
# compute mode of these columns
#replacement_value_dict = df[fixable_num_cols.index].mode().to_dict(orient='records')[0]
#replacement_value_dict

In [ ]:
# replace missing values with replacement values
#df = df.fillna(replacement_value_dict)

In [ ]:
# verify there are no missing values
#df.isnull().sum().value_counts()

What new features can we create, that better capture the information in some of the features?

In [ ]:
#years_sold = df['Yr Sold'] - df['Year Built']
#years_sold[years_sold < 0] # shows there is a negative value; dirty data point

In [ ]:
#years_until_remod = df['Yr Sold'] - df['Year Remod/Add']
#years_until_remod[years_until_remod < 0] # shows three negative values

In [ ]:
# create new columns/features
#df['Years Before Sale'] = years_sold
#df['Years Since Remod'] = years_until_remod

# drop rows with negative values
#df = df.drop([2180,1702,2181], axis=0)

# drop original discrete columns since we've created features
#df = df.drop(['Year Built','Year Remod/Add'], axis=1)

Drop other columns that:
- aren't useful for ML
- leak data about the final sale

In [ ]:
# Drop columns that aren't useful for ML
#df = df.drop(["PID", "Order"], axis=1)

# Drop columns that leak info about the final sale
#df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

# Feature Selection

In [ ]:
# create dataframe with only numerical columns
# numerical_df = transform_df.select_dtypes(include=['int','float'])
# numerical_df.head(5)

In [ ]:
# calculate absolute correlation coefficients
# abs_corr_coeff = numerical_df.corr()['SalePrice'].abs().sort_values()
# abs_corr_coeff

In [ ]:
# Let's only keep columns with a correlation coefficient of larger than 0.4 (arbitrary, worth experimenting later!)
# abs_corr_coeff[abs_corr_coeff > 0.4]


In [ ]:
# Drop columns with less than 0.4 correlation with SalePrice
# transform_df = transform_df.drop(abs_corr_coeff[abs_corr_coeff < 0.4].index, axis=1)

Which categorical columns should we keep?

In [ ]:
# create list of all possible categoricals
# nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
#                     "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
#                     "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
#                     "Misc Feature", "Sale Type", "Sale Condition"]

- Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
- If a categorical column has hundreds of unique values (or categories), should we keep it? When we dummy code this column, hundreds of columns will need to be added back to the data frame.

In [ ]:
# Which categorical columns have we still carried with us? We'll test tehse 
# transform_cat_cols = []
# for col in nominal_features:
#     if col in transform_df.columns:
#         transform_cat_cols.append(col)

# # How many unique values in each categorical column?
# uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
# # Aribtrary cutoff of 10 unique values (worth experimenting)
# drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
# transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [ ]:
# Select just the remaining text columns and convert to categorical
# text_cols = transform_df.select_dtypes(include=['object'])
# for col in text_cols:
#     transform_df[col] = transform_df[col].astype('category')
    
# # Create dummy columns and add back to the dataframe!
# transform_df = pd.concat([
#     transform_df, 
#     pd.get_dummies(transform_df.select_dtypes(include=['category']))
# ], axis=1).drop(text_cols,axis=1)

That's it for the guided steps. Here's some potenial next steps that you can take:

- Continue iteration on feature engineering:
    - Research some other approaches to feature engineering online around housing data.
    - Visit the [Kaggle kernels page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels) for this dataset to see approaches others took.
- Improve your feature selection:
    - Research ways of doing feature selection better with categorical columns (something we didn't cover in this particular course).
